In [2]:
from sklearn.model_selection import train_test_split

from utils.prepare_data import read_data, save_data, target_columns

In [21]:
try:
    seed = snakemake.params["random_seed"]
except AttributeError: # No random seed given
    seed = None

In [9]:
raw = read_data(snakemake.input["sim_params"])
df = raw.loc[raw.sweep_mode.isin(("hard", "rnm (true)", "sgv (true)"))]

In [23]:
sims_per_regime = min(df.sweep_mode.value_counts().min(), snakemake.config["sims_per_regime"])
downsampled = df.groupby("sweep_mode").sample(n=sims_per_regime, random_state=seed)

In [27]:
train, valid = train_test_split(
    downsampled,
    test_size=snakemake.config["validation_percentage"],
    random_state=seed,
    shuffle=True,
    stratify=downsampled.sweep_mode
)

In [8]:
save_data(train, snakemake.output["training"])
save_data(valid, snakemake.output["validation"])